In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
import numpy
import scipy
import matplotlib as mpl
import matplotlib.dates as mpd
import pylab as plt
import datetime
#
import re
#
import json
import netCDF4
#

In [59]:
class NML(dict):
    #
    def __init__(self):
        self.__dict__.update({ky:vl for ky,vl in locals().items() if not ky in ('self', '__class__')})
    
    #
    def assign(self, ky1, ky2, val):
        # a "safe" assignment operator. If the keys do not exist, fail and raise an exception.
        if ky1 in self.keys() and ky2 in self[ky1].keys():
            self[ky1][ky2] = val
            #
            return True
        else:
            err_str = '{} not in self[{}].keys()'.format(ky2, ky1)
            if not(ky1) in self.keys():
                err_str = "{} not in self.keys()"
            #
            raise NameError(err_str)
            return False
    #
    def nml_to_json(self, nml_in=None, json_out=None):
        '''
        # convert nml to dict; save dict as class member; if json_out is not None,
        # export json to json_out
        #
        # NOTE: The format for .nml does not appear to be very strictly defined. Basically, it is not
        #. well defined if key-value pairs are separated by CRLF ('\n') or comma ',' -- oh, and also commas
        #. are allowed (in some cases) withink values. So this might be a work in progress, to be able
        #. to generally define k-v pairs. Right now, its success rests a little bit on, "please, all that
        #. is holy, let them not allow this or that..."
        #
        # @nml_in: filenname of input nml
        # @json_out: filename of output json file.
        #. 
        '''
        #
        if nml_in is None:
            nml_in = self.nml
        #
        #nml_dict = {}
        group_name = ''
        #k_group = 0
        #
        values_out = {}
        ky = ''
        
        with open(nml_in) as fin:
            for rw in fin:
                rw = rw.strip()
                #
                # TODO: maye we should retain comments? if so, we'll need to switch the nested
                #. structure to a list (-like), instead of a dict., to allow for multiple commented-out
                #  entries.
                #
                # for a mid-line comment:
                rw = rw.split('!')[0]
                #
                if len(rw)==0 or rw[0] in ['!', '\n']:
                    continue
                #
                if rw[0]=='&':
                    # new group:
                    if not group_name == '':
                        self[group_name] = values_out
                        #nml_dict[group_name]=values_out
                    #
                    group_name = rw[1:].strip()
                    values_out = {}
                    ky=''
                    val=''
                    #if not group_name in nml_dict.keys():
                    #    nml_dict[group_name]={}
                    continue
                    #
                
                #
                #print('** ', rw)
                #
                # TODO: this logic is almost right, and will probably work most of the time, but
                #. it might be better to be more robust about allowing multi-line entries. Note
                #. this will not work properly for a multi-entry line that end in a multi-line entry.
                if rw.startswith('/'):
                    values_out[ky]=val.strip()
                    continue
                #
                ky_vl = rw.split('=')
                if len(ky_vl) == 1:
                    #print('*** debug: ', val, ky_vl)
                    val = val + ky_vl[0]
                    continue
                elif len(ky_vl) == 2:
                    if not ky=='':
                        values_out[ky]=val.strip()
                    ky,val = [s.strip() for s in ky_vl]
                elif len(ky_vl) > 2:
                    # there are multiple entries, presumably separated by commas?? so
                    # key=val,key=val,key=val...
                    #print('** DEBUG: ', [s.strip() for s in re.split('=|,', rw) if not s.strip()==''])
                    values_out.update(dict(numpy.reshape([s.strip()
                                        for s in re.split('=|,', rw) if not s.strip()==''], (-1,2))))
                
            #
        #
        #self.nml_dict=nml_dict
        #self.update(nml_dict)
        #
        if not json_out is None:
            with open(json_out, 'w') as fout:
                #json.dump(nml_dict, fout)
                # can we just dump self?
                json.dump(self, fout)
                #json.dump({ky:vl for ky,vl in self.items()})
            #
        #
        return None
        #return nml_dict
    #
    
    #
    def json_to_nml(self, nml_out='input.nml', json_in=None, indent=None, file_mode='w'):
        '''
        # convert json or dict to an nml. export to nml_out.
        # TODO: continue to evaluate how lists, tuples, etc. are encoundered and handled. For example,
        #  we want output to be like, format = 6,8 , not format = [6,8]. so far, i don't see any "[]"
        #. characters in .nml files, so we can probably just get rid of them, but it might be smarter
        #. to just recognize when we have a list type... or to enforce that all values are saved internally
        #  as strings... The latter may become necessary, since there does not appear to be a good standard
        #. for comma, space, etc. separating values (or fields).
        '''
        #
        if json_in is None:
            json_in = self
        #
        if indent is None:
            indent = 4*chr(32)
            #
        #
        if isinstance(json_in, str):
            with open(json_in, 'r') as fin:
                json_in = json.load(fin)
            #
        #
        with open(nml_out, file_mode) as fout:
            for group,entries in json_in.items():
                fout.write('&{}\n'.format(group))
                #
                for entry,val in entries.items():
                    fout.write('{}{} = {}\n'.format(indent, entry, val ))
                fout.write('/\n\n')
                
                #
            #
            
                    
class NML_from_nml(NML):
    def __init__(self, input_nml, json_out=None):
        #
        super(NML_from_nml,self).__init__()
        self.__dict__.update({ky:vl for ky,vl in locals().items() if not ky in ('self', '__class__')})
        #
        # is this useful?
        #with open(input_nml) as fin:
        #    self.nml = fin.read()
        #
        self.nml_to_json(input_nml, json_out=json_out)
        
    #
class NML_from_json(NML):
    def __init__(self, input_json, nml_out=None):
        #
        super(NML_from_json,self).__init__()
        self.__dict__.update({ky:vl for ky,vl in locals().items() if not ky in ('self', '__class__')})
        #
        with open(input_json) as fin:
            #self.nml_dict = json.load(fin)
            self.update(json.load(fin))
        #
        if not nml_out is None:
            self.nml = self.json_to_nml(self.nml_json, nml_out)

class AM4_batch_scripter(object):
    mpi_execs = {'mpirun':{'exec': 'mpirun', 'ntasks':'--np ', 'cpu_per_task':'-d '},
                 'srun':{'exec':'srun', 'ntasks':'--ntasks=', 'cpu_per_task':'--cpus-per-task='}}
    #
    def __init__(self, work_dir='workdir', input_data_path='', input_data_tar='',
                 input_nml='input_template.nml', diag_table_src='', force_copy_input=0, do_tar=0,
                 n_tasks=48, n_threads=1):
        # parameters? input data file?
        #
        # not sure what this looks like yet, but... This script/class will be called by a wrapper
        #.  script. This process will constitute a step in a larer script (ie, each ~2 hour run in a
        #. twohour queue process).
        #. script
        # 1) review, set up, etc. the working directory
        # 2) Are the input data there?
        # 3) if not, are the input data availble?
        # 4) if not, is the tar available? if not, get it; then open, then copy.
        # 5) evaluate the input/output data. Have we achieved our objectives
        #.  (which we've not yet defined -- runtime, etc.)? Define restart as necessary 
        #.   (*** though actually, i guess this will be done by the calling script; this script will
        #.    just receive instructions).
        # 6) copy diag_table nd create input.nml
        # 7) execute MPI command
        
        pass
        
    #
    def write_batch_script(self):
        
            
    #
def get_layouts(n_tasks=24):
    '''
    # compute possible layouts. Include (some) error checking for valid n_tasks?
    '''
    # get all integer factor pairs:
    return numpy.array(sorted([(k,int(n_tasks/k)) for k in range(1, int(numpy.ceil(n_tasks**.5)))
                               if n_tasks%k==0],
                                key = lambda rw: numpy.sum(rw)))
#
def get_io_layouts(layout):
    '''
    # AM4 io_layouts. the second "y" term must be an integer factor of the "y" term of the input layout.
    #. Don't yet understand the first term, so for now let's limit it to 1.
    '''
    #
    return numpy.array(sorted([(1,int(layout[1]/k)) for k in range(1, int(numpy.ceil(layout[1]**.5)))
                               if layout[1]%k==0], 
                              key=lambda rw:numpy.sum(rw)))

In [60]:
NML_test = NML_from_nml('input_yoder_v101.nml')
#

# for ky,vl in JJ.nml_dict.items():
#     print('** {}: {}'.format(ky,vl))


for ky in ['coupler_nml','coupler_nml', 'vegn_data_nml', 'simple_sulfate_nml' ]:
    print('*** {}:{}\n'.format(ky, NML_test[ky]))

*** coupler_nml:{'months': '1,', 'days': '0,', 'current_date': '1979,1,1,0,0,0,', 'calendar': "'julian'", 'dt_atmos': '1800,', 'dt_cpld': '7200,', 'use_lag_fluxes': '.true.', 'concurrent': '.false.', 'do_ocean': '.false.', 'ocean_npes': '0', 'atmos_npes': '48', 'atmos_nthreads': '1', 'use_hyper_thread': '.false.', 'ncores_per_node': '24'}

*** coupler_nml:{'months': '1,', 'days': '0,', 'current_date': '1979,1,1,0,0,0,', 'calendar': "'julian'", 'dt_atmos': '1800,', 'dt_cpld': '7200,', 'use_lag_fluxes': '.true.', 'concurrent': '.false.', 'do_ocean': '.false.', 'ocean_npes': '0', 'atmos_npes': '48', 'atmos_nthreads': '1', 'use_hyper_thread': '.false.', 'ncores_per_node': '24'}

*** vegn_data_nml:{'vegn_to_use': "'uniform'", 'K1': '10,', 'K2': '0.1,', 'fsc_liv': '0.9,', 'fsc_wood': '0.45,', 'c1(4)': '0.3', 'c2(4)': '0.3', 'Vmax': '2.0E-5, 2.0E-5, 2.0E-5, 2.0E-5, 1.50E-5,', 'm_cond': '4., 9., 9., 7., 7.,', 'alpha_phot': '0.05, 0.06, 0.06, 0.06, 0.06,', 'gamma_resp': '0.03, 0.02, 0.02, 0.02,

In [61]:
print(NML_test['aerosolrad_package_nml']['sulfate_indices'])

30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,35,35,35,35,35,40,40,40,40,40,45,45,45,45,45,50,50,50,50,50,55,55,55,55,55,60,60,60,60,60,65,65,65,65,65,70,70,70,70,70,75,75,75,75,75,80,80,80,80,82,82,84,84,86,86,88,88,90,91,92,93,94,95,96,97,97,97,97,


### Working NML example.

- Start with a standard template
- Compute layouts for an MPI configuration
- Modify layout variables (in internal JSON/dict)
- Export working input.nml

In [62]:

        
n_tasks = 48
n_threads = 1
#
layouts_1 = get_layouts(n_tasks)
layouts_2 = get_layouts(n_tasks = n_tasks/6)
#
layout_io_1 = get_io_layouts(layouts_1[0])
layout_io_2 = get_io_layouts(layouts_2[0])
#
print('** Layouts_1: ', layouts_1)
print('** Layouts_2: ', layouts_2)
#
print('** Layouts_io_1: ', layout_io_1)
print('** Layouts_io_2: ', layout_io_2)
#
my_nml = NML_from_nml(input_nml='input_yoder_v101.nml')
layout_1=layouts_1[0]
layout_2=layouts_2[0]
#
layout_io = layout_io_1[0]

#
# print out layouts, as they are imported:
for grp in ('fv_core_nml', 'land_model_nml','ocean_model_nml', 'ice_model_nml'):
    print('** {}::layout: {}'.format(grp, my_nml[grp]['layout']))
    print('** {}::io_layout: {}'.format(grp, my_nml[grp]['layout']))
    print('\n')



** Layouts_1:  [[ 6  8]
 [ 4 12]
 [ 3 16]
 [ 2 24]
 [ 1 48]]
** Layouts_2:  [[2 4]
 [1 8]]
** Layouts_io_1:  [[1 4]
 [1 8]]
** Layouts_io_2:  [[1 4]]
** fv_core_nml::layout: 2,4
** fv_core_nml::io_layout: 2,4


** land_model_nml::layout: 2,4
** land_model_nml::io_layout: 2,4


** ocean_model_nml::layout: 4,12
** ocean_model_nml::io_layout: 4,12


** ice_model_nml::layout: 4,12
** ice_model_nml::io_layout: 4,12




In [63]:
print('** ', 48%6)
print('** ', 48%5)

**  0
**  3


In [64]:
#print('** ', my_nml['fv_core_nml']['layout'])
#
for grp in ('fv_core_nml', 'land_model_nml'):
    my_nml.assign(grp, 'layout', ','.join([str(x) for x in layout_2]))
    my_nml.assign(grp, 'io_layout', ','.join([str(x) for x in layout_io]))
    print('** {}:: {}, {}'.format(grp, my_nml[grp]['layout'], my_nml[grp]['io_layout']))
#
for grp in ('ocean_model_nml', 'ice_model_nml'):
    my_nml.assign(grp, 'layout', ','.join([str(x) for x in layout_1]))
    my_nml.assign(grp, 'io_layout', ','.join([str(x) for x in layout_io]))
    print('** {}:: {}, {}'.format(grp, my_nml[grp]['layout'], my_nml[grp]['io_layout']))
#
for ky,vl in [('npx',193), ('npy', 193), ('npz', 50)]:
    my_nml.assign('fv_core_nml', ky, vl)
#
for ky,vl in [('co2_ceiling', 4800.0E-06), ('time_varying_co2', '.true.'), ('co2_base_value',348.0E-06),
            ('co2_floor', 100.0E-06), ('c02_data_source', 'namelist') ]:
    # NOTE: we want to allow new assignment:
    my_nml['radiative_gases_nml'][ky] = vl
print('*** radiative_gases: ', my_nml['radiative_gases_nml'])

** fv_core_nml:: 2,4, 1,4
** land_model_nml:: 2,4, 1,4
** ocean_model_nml:: 6,8, 1,4
** ice_model_nml:: 6,8, 1,4
*** radiative_gases:  {'use_co2_10um': '.true.', 'verbose': '3', 'gas_printout_freq': '240', 'time_varying_co2': '.true.', 'co2_variation_type': "'linear',", 'co2_dataset_entry': '1979 1 1 0 0 0', 'co2_specification_type': "'time_series',", 'co2_floor': 0.0001, 'co2_ceiling': 0.0048, 'co2_data_source': "'input'", 'time_varying_ch4': '.true.,', 'ch4_variation_type': "'linear'", 'ch4_dataset_entry': '1979 1 1 0 0 0', 'ch4_specification_type': "'time_series'", 'ch4_data_source': "'input'", 'time_varying_n2o': '.true.,', 'n2o_variation_type': "'linear'", 'n2o_dataset_entry': '1979 1 1 0 0 0', 'n2o_specification_type': "'time_series'", 'n2o_data_source': "'input'", 'time_varying_f11': '.true.,', 'f11_variation_type': "'linear'", 'f11_dataset_entry': '1979 1 1 0 0 0', 'f11_specification_type': "'time_series'", 'f11_data_source': "'input'", 'time_varying_f12': '.true.,', 'f12_varia

In [65]:
for ky,vl in my_nml['radiative_gases_nml'].items():
    print('** {}:: {}'.format(ky, vl))
print('\n\n')
#
for ky,vl in my_nml['fv_core_nml'].items():
    print('** {}:: {}'.format(ky, vl))

** use_co2_10um:: .true.
** verbose:: 3
** gas_printout_freq:: 240
** time_varying_co2:: .true.
** co2_variation_type:: 'linear',
** co2_dataset_entry:: 1979 1 1 0 0 0
** co2_specification_type:: 'time_series',
** co2_floor:: 0.0001
** co2_ceiling:: 0.0048
** co2_data_source:: 'input'
** time_varying_ch4:: .true.,
** ch4_variation_type:: 'linear'
** ch4_dataset_entry:: 1979 1 1 0 0 0
** ch4_specification_type:: 'time_series'
** ch4_data_source:: 'input'
** time_varying_n2o:: .true.,
** n2o_variation_type:: 'linear'
** n2o_dataset_entry:: 1979 1 1 0 0 0
** n2o_specification_type:: 'time_series'
** n2o_data_source:: 'input'
** time_varying_f11:: .true.,
** f11_variation_type:: 'linear'
** f11_dataset_entry:: 1979 1 1 0 0 0
** f11_specification_type:: 'time_series'
** f11_data_source:: 'input'
** time_varying_f12:: .true.,
** f12_variation_type:: 'linear'
** f12_dataset_entry:: 1979 1 1 0 0 0
** f12_specification_type:: 'time_series'
** f12_data_source:: 'input'
** time_varying_f113:: .tr

In [66]:
my_nml.json_to_nml(nml_out='my_output.nml', json_in=my_nml)

In [67]:
print(','.join([str(x) for x in [1,2,3]]))

1,2,3
